In [8]:
from pyshacl.validate import Validator
from rdflib import OWL, RDF, RDFS, SH, Graph, Namespace

In [9]:
S223 = Namespace("http://data.ashrae.org/standard223#")

In [10]:
g = Graph()
#g.parse('../223standard/models/MODEL_SP223_core-v1.0.ttl')

In [11]:
# Validation function for G36
def run_validation(g, sg):
    v = Validator(
        g,
        shacl_graph=sg,
        options={"iterate_rules": True, "advanced": True},
    )
    conforms, report_graph, report_text = v.run()


    namespace_map = {}
    for prefix, uriref in report_graph.namespaces():
        namespace_map[prefix] = Namespace(uriref)
        # Need OR for result Message
    qs = """
        SELECT ?focusNode ?resultMessage ?resultSeverity ?resultPath
        WHERE {
            BIND(s223:g36 AS ?resultSeverity) .
            ?report rdf:type sh:ValidationReport .
            ?report sh:result ?result .
            ?result sh:focusNode ?focusNode .
            OPTIONAL {?result sh:resultMessage ?resultMessage} .
            ?result sh:resultPath ?resultPath .
            ?result sh:resultSeverity ?resultSeverity .
            }
        """

    # pretty colors
    color_map = {SH.Violation: 33, SH.Info: 34, SH.Warning: 35, S223.g36: 36}

    # query
    results = sorted(report_graph.query(qs, initNs=namespace_map))
    
    prev = None
    for focusNode, resultMessage,resultSeverity, resultPath in results:
        if focusNode != prev:
            print(focusNode)
            prev = focusNode
        color = color_map[resultSeverity]
        print(f"\x1b[{color}m    {resultMessage}\x1b[0m   {resultPath}\x1b[0m")
    return report_text

## Trying out Flow Or and sh:detail
This is random experimentation to get experience writing shacl shapes works. Here I'm testing out using sh:and and sh:or on a simple model file that I've created

In [15]:
file = 'sh-and-or-test.ttl'
g = Graph ()
g.parse(file)
sg = Graph()
sg.parse(file)
v = Validator(
        g,
        shacl_graph=sg,
        options={"iterate_rules": True, "advanced": True},
    )
conforms, report_graph, report_text = v.run()
print(report_text)


<Graph identifier=N8b7f4ad84f79480d8b21f171c0bcb84b (<class 'rdflib.graph.Graph'>)>

## Trying out subclass of carying sh:property constraints
At a high level, we are writing SHACL rules to see when something is a VAV with reheat. If somebody declares that their model is of a VAV with reheat, then this validation rule should be run. It should not however be run for all things of the class VAV, which is why it makes less sense to use sh:targetClass in the long run. 
The main issue we ran into was that SHACL doesn't inherit properties and apply them when sh:targetClass is used. So this means SHACL constraints using sh:targetClass can not inherit properties from their parent classes using rdfs:subClassOf.
However, inheritance can work for sh:properties. If a parent shape is a class (rdfs:Class), and subshapes are rdfs:subclasses of it, then the sh:properties of the parent class will be inherited. The target of this shape must be an INSTANCE of that shape (a, rdf:type) rather than using sh:targetClass. (This makes more sense anyway, because for this project we will be validating specific instances and models, rather than classes.)
so instead of:
:constraint
    a sh:NodeShape;
    sh:targetClass :class .
we should have 
:constraint 
    a sh:NodeShape, rdfs:Class ;
    sh:class :class
:instance
    a :class ;
    a :constraint ;
.

This could work by ADDING the triple ":instance a :constraint" when validation is run. 
so, before validation the instance is just a :class 
:instance 
    a:class .
then before validation is run, the instance is targeted by further describing what it should be (something satisfying :constraint).
the instance is made 
:instance 
    a :class, :constraint .


In [7]:
file = 'gen-test.ttl'
g = Graph ()
g.parse(file)
sg = Graph()
sg.parse(file)
v = Validator(
        g,
        shacl_graph=sg,
        options={"iterate_rules": True, "advanced": True},
    )
conforms, report_graph, report_text = v.run()
print(report_text)


Validation Report
Conforms: False
Results (2):
Constraint Violation in ClassConstraintComponent (http://www.w3.org/ns/shacl#ClassConstraintComponent):
	Severity: sh:Violation
	Source Shape: test:constraint
	Focus Node: test:invalid_class_type
	Value Node: test:invalid_class_type
	Message: constraint applied to an invalid class
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path test:test_prop ]
	Focus Node: test:instance
	Result Path: test:test_prop
	Message: Less than 1 values on test:instance->test:test_prop

